In [1]:
%%capture
!pip install selenium

In [2]:
%%capture
!wget https://chromedriver.storage.googleapis.com/84.0.4147.30/chromedriver_linux64.zip

In [3]:
%%capture
!unzip chromedriver_linux64.zip

In [4]:
%%capture
!pip install pyserial

In [5]:
from selenium import webdriver
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import configparser
import serial
import time
from IPython.display import Image

Read in the configuration file for OLX account credentials and chrome path.

In [6]:
# Read in the credentials for olx account and chrome
read_config = configparser.ConfigParser()
read_config.read("credentials.ini")

phone = int(read_config.get("OLX", "phone"))
password = str(read_config.get("OLX", "pass"))
chrome_path = str(read_config.get("CHROME", "path"))

We can log-in to the OLX account using either phone or gmail or user and password. Trying to log-in via gmail causes the following issue for which I couldn't find a workaround:

In [7]:
Image(url='couldntSignIn.png', width=500)

Thus, I tried logging in via phone instead, and it worked just fine!

We load connect our machine to an arduino using a USB, and upload _PhysicalPixel.ino_ from 'Examples' in Arduino IDE to the arduino board. We use selenium to login into our OLX account and then navigate to the chat page and see if there are any unread chats. If so, we'll write the letter 'H' in bytes to our arduino serial, which results in LED pin 13 of the board to turn on. Else we write an 'L', which causes the LED to turn off.

In [8]:
cap = DesiredCapabilities.CHROME
cap['chrome.switches'] = ['--disable-local-storage']
options = webdriver.ChromeOptions()
options.add_argument("--user-data-dir="+chrome_path)
options.add_argument("--disable-extensions")
options.add_argument("--incognito");
options.add_argument("--set-headless")
chrome_driver =os.getcwd() + '/chromedriver'
os.environ["webdriver.chrome.driver"] = chrome_driver

while(True):
    with serial.Serial('/dev/ttyACM0', 9800, timeout=1) as ser:
        try:
            driver = webdriver.Chrome(executable_path=chrome_driver,options=options, desired_capabilities=cap)
            #Now go to olx page, click on login
            driver.get("https://www.olx.in/")
            #Login:
            login_button = driver.find_element_by_xpath("//button[.//span[text()='Login']]")
            login_button.click()
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.XPATH,"//button[.//span[text()='Continue with Phone']]")))

            driver.find_element_by_xpath("//button[.//span[text()='Continue with Phone']]").click()
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.NAME,'phone')))

            driver.find_element_by_name('phone').send_keys(phone)
            driver.implicitly_wait(5)
            driver.find_element_by_xpath("//button[.//span[text()='Next']]").click()
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.NAME,'password')))

            driver.find_element_by_name('password').send_keys(password)
            driver.implicitly_wait(5)
            driver.find_element_by_xpath('//button[.//span[text()="Log in"]]').click()
            wait.until(EC.presence_of_element_located((By.XPATH,"//div[@data-aut-id='iconProfile']")))


            #Now we are logged in!
            driver.get('https://www.olx.in/nf/chat')
            wait.until(EC.presence_of_element_located((By.XPATH,"//span[.//span[text()='Inbox']]")))
            unread_count_tag = driver.find_elements_by_xpath("//span[.//span[text()='Inbox']]/following-sibling::div/span")
            if(len(unread_count_tag) == 0):
                print(f'You have no new messages!')
                ser.write(b'L')
                time.sleep(1)
            else:
                print(f'You have {unread_count_tag[0].text} new messages!')
                ser.write(b'H')
                time.sleep(1)

        except Exception as e:
            print(e)
        finally:
            driver.close()

SerialException: [Errno 2] could not open port /dev/ttyACM0: [Errno 2] No such file or directory: '/dev/ttyACM0'

**What more can be done here**:
1. Upload the code to arduino board to see if it can run standalone.
2. Implement the same using image processing to see if that's faster.